In [ ]:
import os
import json
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras.layers import (
    Dropout, Activation, Input, Conv2D, BatchNormalization, ReLU,
    MaxPooling2D, UpSampling2D, Concatenate, Multiply, Add
)
from tensorflow.keras.models import ModelQ
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.losses import Loss
import supervisely as sly
from sklearn.model_selection import train_test_split
from skimage.metrics import adapted_rand_error
import matplotlib.pyplot as plt
from tqdm import tqdm
import keras
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from scipy import ndimage
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# pielāgošana
class Config:
    IMAGE_DIR = "/mnt/c/dataset_folder/ds/img/"
    MASK_DIR = "/mnt/c/dataset_folder/ds/ann/"
    MODEL_DIR = "/mnt/c/saved_models/"
    IMG_SIZE = (256, 256)
    BATCH = 3
    AUTO_THRESH = False

os.makedirs(Config.MODEL_DIR, exist_ok=True)

In [ ]:
# Datu ielāde
def load_images_and_masks(img_dir, mask_dir, meta, target_size):
    images, masks = [], []
    for img_name in tqdm(os.listdir(img_dir), desc="Loading images"):
        if not img_name.endswith(".png"): continue
        
        img_path = os.path.join(img_dir, img_name)
        orig_img = sly.image.read(img_path)
        h, w = orig_img.shape[:2]
        mask = np.zeros((h, w), dtype=np.uint8)
        mask_path = os.path.join(mask_dir, img_name + ".json")
        if os.path.exists(mask_path):
            with open(mask_path) as f:
                ann = sly.Annotation.from_json(json.load(f), meta)
            for label in ann.labels:
                try:
                    label.draw(mask, color=255)
                except Exception as e:
                    print(f"Couldn't draw label on {img_name}: {str(e)}")
                    continue

        image = cv2.resize(orig_img, target_size) / 255.0
        mask = cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST)
        
        masks.append(mask.astype(np.float32)[..., None] / 255.0)
        images.append(image)
    
    return np.array(images), np.array(masks)

with open(os.path.join(Config.MASK_DIR, "meta.json")) as f:
    meta = sly.ProjectMeta.from_json(json.load(f))

images, masks = load_images_and_masks(Config.IMAGE_DIR, Config.MASK_DIR, meta, Config.IMG_SIZE)
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

In [ ]:
# Modeļa arhitektūra
def unet_model(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)
    
    e1 = Conv2D(32, 3, padding='same')(inputs)
    e1 = BatchNormalization()(e1)
    e1 = Activation('relu')(e1)
    e1 = Conv2D(32, 3, padding='same')(e1)
    e1 = BatchNormalization()(e1)
    e1 = Activation('relu')(e1)
    p1 = MaxPooling2D()(e1)
    p1 = Dropout(0.1)(p1)
    
    e2 = Conv2D(64, 3, padding='same')(p1)
    e2 = BatchNormalization()(e2)
    e2 = Activation('relu')(e2)
    e2 = Conv2D(64, 3, padding='same')(e2)
    e2 = BatchNormalization()(e2)
    e2 = Activation('relu')(e2)
    p2 = MaxPooling2D()(e2)
    p2 = Dropout(0.2)(p2)
    
    e3 = Conv2D(128, 3, padding='same')(p2)
    e3 = BatchNormalization()(e3)
    e3 = Activation('relu')(e3)
    e3 = Conv2D(128, 3, padding='same')(e3)
    e3 = BatchNormalization()(e3)
    e3 = Activation('relu')(e3)
    p3 = MaxPooling2D()(e3)
    p3 = Dropout(0.3)(p3)
    
    m = Conv2D(256, 3, dilation_rate=2, padding='same')(p3)
    m = BatchNormalization()(m)
    m = Activation('relu')(m)
    m = Conv2D(256, 3, dilation_rate=2, padding='same')(m)
    m = BatchNormalization()(m)
    m = Activation('relu')(m)
    m = Dropout(0.4)(m)
    
    d1 = UpSampling2D()(m)
    d1 = Conv2D(128, 2, padding='same')(d1)
    d1 = BatchNormalization()(d1)
    d1 = Activation('relu')(d1)
    d1 = Concatenate()([d1, e3])
    d1 = Conv2D(128, 3, padding='same')(d1)
    d1 = BatchNormalization()(d1)
    d1 = Activation('relu')(d1)
    d1 = Conv2D(128, 3, padding='same')(d1)
    d1 = BatchNormalization()(d1)
    d1 = Activation('relu')(d1)
    
    d2 = UpSampling2D()(d1)
    d2 = Conv2D(64, 2, padding='same')(d2)
    d2 = BatchNormalization()(d2)
    d2 = Activation('relu')(d2)
    d2 = Concatenate()([d2, e2])
    d2 = Conv2D(64, 3, padding='same')(d2)
    d2 = BatchNormalization()(d2)
    d2 = Activation('relu')(d2)
    d2 = Conv2D(64, 3, padding='same')(d2)
    d2 = BatchNormalization()(d2)
    d2 = Activation('relu')(d2)
    
    d3 = UpSampling2D()(d2)
    d3 = Conv2D(32, 2, padding='same')(d3)
    d3 = BatchNormalization()(d3)
    d3 = Activation('relu')(d3)
    d3 = Concatenate()([d3, e1])
    d3 = Conv2D(32, 3, padding='same')(d3)
    d3 = BatchNormalization()(d3)
    d3 = Activation('relu')(d3)
    d3 = Conv2D(32, 3, padding='same')(d3)
    d3 = BatchNormalization()(d3)
    d3 = Activation('relu')(d3)
    
    attention = Conv2D(1, 1, activation='sigmoid')(d3)
    d3 = Multiply()([d3, attention])
    
    outputs = Conv2D(1, 1, activation='sigmoid')(d3)
    
    return Model(inputs, outputs)

class FocusedSegmentationLoss(Loss):
    def __init__(self, alpha=0.8, beta=0.2, gamma=1.5, name='focused_loss'):
        super().__init__(name=name)
        self.alpha = tf.constant(alpha, dtype=tf.float32)
        self.beta = tf.constant(beta, dtype=tf.float32)
        self.gamma = tf.constant(gamma, dtype=tf.float32)
        self.smooth = tf.constant(1e-6, dtype=tf.float32)

    def call(self, y_true, y_pred):
        tp = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
        fp = tf.reduce_sum((1 - y_true) * y_pred, axis=[1, 2, 3])
        fn = tf.reduce_sum(y_true * (1 - y_pred), axis=[1, 2, 3])
        tversky = (tp + self.smooth) / (tp + self.alpha * fp + self.beta * fn + self.smooth)
        focal_tversky = tf.pow(1 - tversky, self.gamma)
        
        if len(y_true.shape) == 4:
            y_true_dx, y_true_dy = tf.image.image_gradients(y_true)
            y_pred_dx, y_pred_dy = tf.image.image_gradients(y_pred)
            boundary_loss = tf.reduce_mean(
                tf.abs(y_true_dx - y_pred_dx) + tf.abs(y_true_dy - y_pred_dy),
                axis=[1, 2, 3]
            )
        else:
            boundary_loss = 0.0
        
        return focal_tversky + 0.05 * boundary_loss

    def get_config(self):
        return {'alpha': float(self.alpha), 'beta': float(self.beta), 'gamma': float(self.gamma)}

model = unet_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=FocusedSegmentationLoss(alpha=0.9, beta=0.1, gamma=1.5),
    metrics=['accuracy', tf.keras.metrics.MeanIoU(2)]
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(Config.MODEL_DIR, 'Segment_modelis_1.keras'),
    save_best_only=True,
    monitor='val_loss'
)

try:
    model.fit(
        tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(Config.BATCH),
        validation_data=(X_val, y_val),
        epochs=25,
        callbacks=[checkpoint]
    )
except tf.errors.ResourceExhaustedError:
    print("Atmiņas trūkums.")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

class SegmentationEvaluator:
    def __init__(self, model, X_test, y_test):
        self.model = model
        self.X_test = X_test
        self.y_test = y_test
        self.threshold = 0.1  # Default threshold
        self.min_object_size = 100  # Minimum pixels to consider an object
    
    def calculate_iou(self, y_true, y_pred):
        intersection = np.sum(y_true * y_pred)
        union = np.sum(y_true) + np.sum(y_pred) - intersection
        return intersection / (union + 1e-7)
    
    def find_optimal_threshold(self, n_samples=100):
        preds = self.model.predict(self.X_test[:n_samples], verbose=0)
        thresholds = np.linspace(0.1, 0.9, 20)
        best_thresh = 0.5
        best_dice = 0
        
        for thresh in thresholds:
            y_true_binary = (self.y_test[:n_samples] > thresh).astype(np.int32)
            y_pred_binary = (preds > thresh).astype(np.int32)
            dice = f1_score(y_true_binary.flatten(), y_pred_binary.flatten())
            
            if dice > best_dice:
                best_dice = dice
                best_thresh = thresh
        
        self.threshold = best_thresh
        return best_thresh
    
    def _get_connected_components(self, mask):
        labeled, num_features = ndimage.label(mask)
        objects = []
        for i in range(1, num_features+1):
            obj_mask = (labeled == i)
            if np.sum(obj_mask) >= self.min_object_size:
                objects.append(obj_mask)
        return objects
    
    def evaluate(self):
        preds = self.model.predict(self.X_test, verbose=0)
        
        y_true_binary = (self.y_test > self.threshold).astype(np.int32)
        y_pred_binary = (preds > self.threshold).astype(np.int32)
        
        tp = np.sum(y_true_binary * y_pred_binary)  # True positives
        fp = np.sum((1-y_true_binary) * y_pred_binary)  # False positives
        fn = np.sum(y_true_binary * (1-y_pred_binary))  # False negatives
        tn = np.sum((1-y_true_binary) * (1-y_pred_binary))  # True negatives
        
        false_detections = 0
        missed_detections = 0
        good_detections = 0
        
        for i in range(len(self.X_test)):
            true_objects = self._get_connected_components(y_true_binary[i])
            pred_objects = self._get_connected_components(y_pred_binary[i])
            
            if len(true_objects) == 0:
                false_detections += len(pred_objects)
            
            else:
                if len(pred_objects) == 0:
                    missed_detections += len(true_objects)
                else:
                    for true_obj in true_objects:
                        max_iou = 0
                        for pred_obj in pred_objects:
                            iou = self.calculate_iou(true_obj, pred_obj)
                            if iou > max_iou:
                                max_iou = iou
                        if max_iou > 0.5:
                            good_detections += 1
                        else:
                            missed_detections += 1
                    
                    false_detections += max(0, len(pred_objects) - len(true_objects))
        
        total_objects = np.sum([len(self._get_connected_components(m)) for m in y_true_binary])
        
        return {
            'pixel_accuracy': accuracy_score(y_true_binary.flatten(), y_pred_binary.flatten()),
            'precision': precision_score(y_true_binary.flatten(), y_pred_binary.flatten()),
            'recall': recall_score(y_true_binary.flatten(), y_pred_binary.flatten()),
            'IoU': self.calculate_iou(y_true_binary, y_pred_binary),
            'Dice': f1_score(y_true_binary.flatten(), y_pred_binary.flatten()),
            
            'true_positives': tp,
            'false_positives': fp,
            'true_negatives': tn,
            'false_negatives': fn,
            
            'object_detection_rate': good_detections / (total_objects + 1e-7),
            'false_discovery_rate': false_detections / (false_detections + good_detections + 1e-7),
            'missed_object_rate': missed_detections / (total_objects + 1e-7),
            'total_objects': total_objects,
            'false_detections': false_detections,
            'missed_detections': missed_detections,
            'good_detections': good_detections,
            
            'threshold': self.threshold
        }
    
    def visualize_samples(self, n_samples=3, show_analysis=False):
        preds = self.model.predict(self.X_test[:n_samples], verbose=0)
        
        plt.figure(figsize=(18, 5*n_samples))
        for i in range(n_samples):
            y_true_bin = (self.y_test[i] > self.threshold).astype(np.int32)
            y_pred_bin = (preds[i] > self.threshold).astype(np.int32)
            
            plt.subplot(n_samples, 4, i*4+1)
            plt.imshow(self.X_test[i])
            plt.title(f"Image {i+1}")
            plt.axis('off')
            
            plt.subplot(n_samples, 4, i*4+2)
            plt.imshow(y_true_bin.squeeze(), cmap='gray')
            plt.title("True Mask")
            plt.axis('off')
            
            plt.subplot(n_samples, 4, i*4+3)
            plt.imshow(y_pred_bin.squeeze(), cmap='gray')
            plt.title(f"Predicted (Threshold={self.threshold:.2f})")
            plt.axis('off')
            
            if show_analysis:
                plt.subplot(n_samples, 4, i*4+4)
                error_mask = np.zeros_like(y_true_bin)
                error_mask[(y_true_bin==1)&(y_pred_bin==0)] = 1  
                error_mask[(y_true_bin==0)&(y_pred_bin==1)] = 2  
                error_mask[(y_true_bin==1)&(y_pred_bin==1)] = 3  
                
                plt.imshow(error_mask.squeeze(), cmap='jet', vmin=0, vmax=3)
                plt.title("Errors: FN(LBlue), TN(Blue), FP(Yellow), TP(Red)")
                plt.axis('off')
                plt.colorbar(ticks=[0,1,2,3], label='Error Type')
        
        plt.tight_layout()
        plt.show()

evaluator = SegmentationEvaluator(model, X_test, y_test)
print("Optimal threshold:", evaluator.find_optimal_threshold())
metrics = evaluator.evaluate()

print("\nDetailed Metrics:")
print(f"Pixel Accuracy: {metrics['pixel_accuracy']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"IoU: {metrics['IoU']:.4f}")
print(f"Dice: {metrics['Dice']:.4f}")

print("\nPixel Counts:")
print(f"True Positives: {metrics['true_positives']}")
print(f"False Positives: {metrics['false_positives']}")
print(f"True Negatives: {metrics['true_negatives']}")
print(f"False Negatives: {metrics['false_negatives']}")

print("\nObject Detection:")
print(f"Total Objects: {metrics['total_objects']}")
print(f"Good Detections: {metrics['good_detections']}")
print(f"Missed Detections: {metrics['missed_detections']}")
print(f"False Detections: {metrics['false_detections']}")
print(f"Detection Rate: {metrics['object_detection_rate']:.2%}")
print(f"False Discovery Rate: {metrics['false_discovery_rate']:.2%}")

evaluator.visualize_samples(n_samples=100, show_analysis=True) #piemēru daudzums